In [1]:
# - 원래는 멀티턴 대화를 위해 매번 GPT와 사용자의 대화내용을 리스트나 딕셔너리에 추가하는 코드를 작성해야 했음
# - 랭체인의 히스토리 기능을 사용하면 멀티턴 대화를 더 쉽게 구현 가능

from langchain_core.chat_history import InMemoryChatMessageHistory  # 메모리에 대화 기록을 저장하는 클래스(리스트 형태로 보관. 어플리케이션을 종료하면 대화내용이 사라지기 때문에 계속 사용하고 싶다면 파일이나 DB에 저장해야 함
from langchain_core.runnables.history import RunnableWithMessageHistory  # 메시지 기록을 활용해 실행 가능한 래퍼wrapper 클래스
#모델을 생성할 때 대화 기록을 함께 전달할 수 있도록 하는 클래스
from langchain_openai import ChatOpenAI  # 오픈AI 모델을 사용하는 랭체인 챗봇 클래스
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-4o-mini") #랭체인에서 사용할 모델 정의

# 세션id 별 사용자 대화 기록을 저장할 딕셔너리
store = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # 메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id]  # 해당 세션의 대화 기록을 반환

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성 (대화 내용을 저장하면서 대화를 이어 나갈 수 있음)
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [2]:
config = {"configurable": {"session_id": "abc2"}}  # 세션 ID를 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 이성용이야.")],
    config=config,
)

print(response.content)

안녕하세요, 이성용님! 어떻게 도와드릴까요?


In [3]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 이성용입니다. 다른 질문이 있으신가요?


In [4]:
config = {"configurable": {"session_id": "abc3"}} #세션 아이디를 새로하면  abc2에 기록했던 대화 내용은 전혀 없는 상태

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

response.content

'죄송하지만, 당신의 이름을 알 수 있는 정보가 없습니다. 당신의 이름을 알려주시면 좋을 것 같아요!'

In [5]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config,
)

response.content

'우리는 당신의 이름에 대해 이야기했습니다. 당신의 이름은 이성용이라고 말씀하셨습니다. 더 궁금한 것이 있으면 말씀해 주세요!'

In [7]:
#스트림 방식으로 출력하기

config = {"configurable": {"session_id": "abc2"}}
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
):
    print(r.content, end="|")

|이|성|용|이라는| 이름|을| 고려|했|을| 때|,| 한국|에서| 오|신| 분|일| 가능|성이| 높|습니다|.| 한국|의| 문화|에| 대해| 간|단|히| 소개|해|드|리|겠습니다|.

|1|.| **|전|통|과| 현대|**|:| 한국|은| 고|유|의| 전|통| 문|화를| 가지고| 있으며|,| 사|찰|,| 궁|궐|과| 같은| 역사|적인| 장소|들이| 많이| 있습니다|.| 동시에| 현대|적|이고| 발전|된| 도시| 환경|도| 갖|추|고| 있습니다|.

|2|.| **|음|식|**|:| 한국| 음식|은| 다양|하고| 맛|있|으며|,| 김|치|,| 비|빔|밥|,| 불|고|기| 등이| 유명|합니다|.| 한국| 식|문화|에서는| 공유|하는| 식|사가| 중요|시|됩니다|.

|3|.| **|명|절|과| 축|제|**|:| 설|날|과| 추|석| 같은| 전|통| 명|절|이| 있으며|,| 가족|들이| 모|여| 조|상|에게| 제|사를| 지|내|고|,| 함께| 음|식을| 나|누|는| 문화|가| 있습니다|.

|4|.| **|예|술|과| 공연|**|:| K|-pop|과| 한국| 드|라마|는| 전| 세계|적으로| 인|기를| 끌|고| 있으며|,| 전|통| 음악|인| 국|악|과| 무|용|도| 중요한| 문화|유|산|입니다|.

|5|.| **|한|류|**|:| 최근| 몇| 년|간| 한|류|가| 세계|적으로| 확|산|되|면서| 한국|의| 음악|,| 드|라마|,| 패|션| 등이| 많은| 사랑|을| 받고| 있습니다|.

|더| 알고| 싶|으|신| 부분|이| 있다|면| 말씀|해| 주세요|!||